1. Buatlah sebuah Sistem REST API dengan menggunakan dua data di atas
2. Buatlah 1 tabel tambahan yang mendukung backend aplikasi kalian berdasar pada dua berkas di atas. (Silahkan masukkan 3 tabel tersebut dalam MySQL lokal dan berkreasi dengannya)
3. Beberapa berkas yang harus ada
a. models
b. router
4. Untuk berkas lain jika dirasa dibutuhkan silahkan tambahkan, misal schema, berkas utama (baik diberi nama main atau app), dll (konfigurasi .env sesuaikan dengan materi)
5. Setiap Tabel harus ada relasi dengan minimal satu tabel lainnya.
6. Gunakan ORM
7. Silahkan memilih menggunakan Flask atau FastAPI
8. Ada beberapa endpoint api yang harus ada
9. Bisa melihat semua barang
10. Bisa melihat barang dengan kata kunci (jika dicari buku, maka semua barang yang ada kata "buku" akan masuk pencarian)
11. Silahkan tambahkan endpoint lain sesuai kebutuhan
12. Aplikasikan penggunaan JWT, contohnya user hanya bisa melihat data transaksi user tersebut tapi tidak user lain (hanya contoh). product tidak perlu JWT (karena semua user bisa melihat semua product)
13. Dokumetasikan API kalian dengan Swagger
14. Semua tambahan dari ketentuan di atas akan menjadi nilai tambah
15. Jelaskan tentang aplikasi backend kalian pada readme.md

In [1]:
import pandas as pd

df_users = pd.read_csv('users.csv')

display(df_users)

,customer_id,name,city,state,postal
0,1,Claire Gute,Fremont,Nebraska,68025
1,2,Sean O'Donnell,Provo,Utah,84604
2,3,Brosina Hoffman,Keller,Texas,76248
3,4,Andrew Allen,Harlingen,Texas,78550
4,5,Irene Maddox,Kenner,Louisiana,70065
...,...,...,...,...,...
404,405,Ionia McGrath,Plainfield,New Jersey,7060
405,406,Anemone Ratner,Mission Viejo,California,92691
406,407,Sung Chung,Abilene,Texas,79605
407,408,Ricardo Emerson,Jackson,Michigan,49201


In [2]:
df_products = pd.read_csv('products.csv', sep=';', encoding='latin-1')
df_products.columns = ['product_id', 'product_name', 'category', 'sub_category']
display(df_products)

,product_id,product_name,category,sub_category
0,1,Bush Somerset Collection Bookcase,Furniture,Bookcases
1,2,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs
2,3,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,Labels
3,4,Bretford CR4500 Series Slim Rectangular Table,Furniture,Tables
4,5,Eldon Fold 'N Roll Cart System,Office Supplies,Storage
...,...,...,...,...
1845,1846,RCA ViSYS 25425RE1 Corded phone,Technology,Phones
1846,1847,Cisco 8961 IP Phone Charcoal,Technology,Machines
1847,1848,"Eureka Hand Vacuum, Bagless",Office Supplies,Appliances
1848,1849,LG G2,Technology,Phones


In [3]:
import random
from datetime import datetime, timedelta

random.seed(7)

def random_tanggal():
    # Set the start and end dates
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2022, 12, 17)
    # Generate a random date between the start and end dates
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    # Convert the random date to a string in the DD--MM-YY format
    formatted_date = random_date.strftime("%d-%m-20%y")
    return formatted_date

n = 1000 #banyaknya kolom

df_purchase = pd.DataFrame({'date':[random_tanggal() for i in range(n)],
                         'name':[random.choices(df_users['name'])[0] for i in range(n)], 
                         'product_name':[random.choices(df_products['product_name'])[0] for i in range(n)],
                         'quantity':[random.randint(1,25) for i in range(n)]
                        })
display(df_purchase)

,date,name,product_name,quantity
0,25-10-2021,Matt Collins,Bevis 44 x 96 Conference Tables,14
1,04-11-2020,Lena Cacioppo,HTC One Mini,8
2,19-03-2022,Max Jones,Acco Suede Grain Vinyl Round Ring Binder,8
3,08-04-2020,Dan Lawera,Global Deluxe High-Back Office Chair in Storm,12
4,28-05-2020,Liz Preis,"Avery Trapezoid Ring Binder, 3"" Capacity, Blac...",7
...,...,...,...,...
995,01-10-2021,Carl Ludwig,Boston 16765 Mini Stand Up Battery Pencil Shar...,2
996,08-06-2020,Joy Daniels,"IBM Multi-Purpose Copy Paper, 8 1/2 x 11"", Case",22
997,20-02-2021,Ralph Ritter,"Fellowes Premier Superior Surge Suppressor, 10...",4
998,05-03-2020,Shui Tom,Martin Yale Chadless Opener Electric Letter Op...,18


In [4]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from dotenv import dotenv_values
import urllib.parse

params = {'db':dotenv_values('TugasAkhir.env')}

db_user = params['db']['MYSQL_USERNAME']
db_password = urllib.parse.quote_plus(params['db']['MYSQL_PASSWORD'])
db_host = params['db']['MYSQL_HOST']
db_port = params['db']['MYSQL_PORT']
db_name = params['db']['MYSQL_DB']

# Buat koneksi ke database MySQL
SQLALCHEMY_DATABASE_URL = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(SQLALCHEMY_DATABASE_URL, echo=True, pool_pre_ping=True)

# Buat base class untuk ORM
Base = declarative_base()

try:
    # Buat tabel products
    class Products(Base):
        __tablename__ = 'products'
        product_id = Column(Integer, nullable=False)
        product_name = Column(String(50), primary_key=True)
        category = Column(String(50), nullable=False)
        sub_category = Column(String(50), nullable=False)
    
    # Buat tabel users
    class Users(Base):
        __tablename__ = 'users'
        customer_id = Column(Integer, nullable=False)
        name = Column(String(50), primary_key=True)
        city = Column(String(50), nullable=False)
        state = Column(String(50), nullable=False)
        postal = Column(Integer, nullable=False)

    # Buat tabel purchase
    class Purchase(Base):
        __tablename__ = 'purchase'
        name = Column(String(50), nullable=False, primary_key=True)
        date = Column(DateTime, nullable=False)
        product_name = Column(String(50), nullable=False, primary_key=True)
        quantity = Column(Integer, nullable=False)
    
    Base.metadata.create_all(engine)
except Exception as e:
    print(e)    

2022-12-23 05:35:09,331 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-12-23 05:35:09,333 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 05:35:09,335 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-12-23 05:35:09,336 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 05:35:09,338 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-12-23 05:35:09,339 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 05:35:09,341 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-23 05:35:09,343 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 05:35:09,343 INFO sqlalchemy.engine.Engine [generated in 0.00098s] {'table_schema': 'penjualan_uzu', 'table_name': 'products'}
2022-12-23 05:35:09,346 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 05:35:0

In [5]:
# Insert data ke tabel products
df_products.to_sql('products', engine, if_exists='replace', index=False)

# Insert data ke tabel users
df_users.to_sql('users', engine, if_exists='replace', index=False)

# Insert data ke tabel purchase
df_purchase.to_sql('purchase', engine, if_exists='replace', index=False)

2022-12-23 05:35:09,394 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 05:35:09,394 INFO sqlalchemy.engine.Engine [cached since 0.05192s ago] {'table_schema': 'penjualan_uzu', 'table_name': 'products'}
2022-12-23 05:35:09,398 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 05:35:09,399 INFO sqlalchemy.engine.Engine [cached since 0.057s ago] {'table_schema': 'penjualan_uzu', 'table_name': 'products'}
2022-12-23 05:35:09,403 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `penjualan_uzu`
2022-12-23 05:35:09,404 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 05:35:09,416 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `products`
2022-12-23 05:35:09,417 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 05:35:09,422 INFO sqlalchemy.engine.Engine BEGIN (implic

-1

In [6]:
# Buat Session
Session = sessionmaker(bind=engine)
session = Session()
conn = engine.connect()

In [7]:
conn.execute('select * from users').all()

2022-12-23 05:35:11,306 INFO sqlalchemy.engine.Engine select * from users
2022-12-23 05:35:11,306 INFO sqlalchemy.engine.Engine [raw sql] {}


[(1, 'Claire Gute', 'Fremont', 'Nebraska', 68025),
 (2, "Sean O'Donnell", 'Provo', 'Utah', 84604),
 (3, 'Brosina Hoffman', 'Keller', 'Texas', 76248),
 (4, 'Andrew Allen', 'Harlingen', 'Texas', 78550),
 (5, 'Irene Maddox', 'Kenner', 'Louisiana', 70065),
 (6, 'Pete Kriz', 'Santa Ana', 'California', 92704),
 (7, 'Alejandro Grove', 'Pharr', 'Texas', 78577),
 (8, 'Zuschuss Donatelli', 'Taylor', 'Michigan', 48180),
 (9, 'Sandra Flanagan', 'New Rochelle', 'New York', 10801),
 (10, 'Emily Burns', 'Oswego', 'Illinois', 60543),
 (11, 'Eric Hoffmann', 'Santa Barbara', 'California', 93101),
 (12, 'Tracy Blumstein', 'Lorain', 'Ohio', 44052),
 (13, "Patrick O'Donnell", 'Bloomington', 'Illinois', 61701),
 (14, 'Lena Hernandez', 'Atlanta', 'Georgia', 30318),
 (15, 'Darren Powers', 'Moreno Valley', 'California', 92553),
 (16, 'Ted Butterfield', 'Redlands', 'California', 92374),
 (17, 'Kunst Miller', 'Philadelphia', 'Pennsylvania', 19140),
 (18, 'Karen Daniels', 'Park Ridge', 'Illinois', 60068),
 (19, '

In [8]:
conn.execute('select * from products').all()

2022-12-23 05:35:11,630 INFO sqlalchemy.engine.Engine select * from products
2022-12-23 05:35:11,631 INFO sqlalchemy.engine.Engine [raw sql] {}


[(1, 'Bush Somerset Collection Bookcase', 'Furniture', 'Bookcases'),
 (2, 'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back', 'Furniture', 'Chairs'),
 (3, 'Self-Adhesive Address Labels for Typewriters by Universal', 'Office Supplies', 'Labels'),
 (4, 'Bretford CR4500 Series Slim Rectangular Table', 'Furniture', 'Tables'),
 (5, "Eldon Fold 'N Roll Cart System", 'Office Supplies', 'Storage'),
 (6, 'Eldon Expressions Wood and Plastic Desk Accessories, Cherry Wood', 'Furniture', 'Furnishings'),
 (7, 'Newell 322', 'Office Supplies', 'Art'),
 (8, 'Mitel 5320 IP Phone VoIP phone', 'Technology', 'Phones'),
 (9, 'DXL Angle-View Binders with Locking Rings by Samsill', 'Office Supplies', 'Binders'),
 (10, 'Belkin F5C206VTEL 6 Outlet Surge', 'Office Supplies', 'Appliances'),
 (11, 'Chromcraft Rectangular Conference Tables', 'Furniture', 'Tables'),
 (12, 'Konftel 250 Conference\xa0phone\xa0- Charcoal black', 'Technology', 'Phones'),
 (13, 'Xerox 1967', 'Office Supplies', 'Paper'),
 (14, 

In [9]:
conn.execute('select * from purchase').all()

2022-12-23 05:35:11,911 INFO sqlalchemy.engine.Engine select * from purchase
2022-12-23 05:35:11,912 INFO sqlalchemy.engine.Engine [raw sql] {}


[('25-10-2021', 'Matt Collins', 'Bevis 44 x 96 Conference Tables', 14),
 ('04-11-2020', 'Lena Cacioppo', 'HTC One Mini', 8),
 ('19-03-2022', 'Max Jones', 'Acco Suede Grain Vinyl Round Ring Binder', 8),
 ('08-04-2020', 'Dan Lawera', 'Global Deluxe High-Back Office Chair in Storm', 12),
 ('28-05-2020', 'Liz Preis', 'Avery Trapezoid Ring Binder, 3" Capacity, Black, 1040 sheets', 7),
 ('11-07-2020', 'James Galang', 'Tennsco Commercial Shelving', 11),
 ('18-01-2022', 'Nora Paige', '3-ring staple pack', 25),
 ('28-04-2020', 'Doug Bickford', 'Elite 5" Scissors', 14),
 ('05-11-2022', 'Adam Shillingsburg', '3M Polarizing Light Filter Sleeves', 21),
 ('15-03-2021', 'Carlos Daly', 'Avery Arch Ring Binders', 9),
 ('17-03-2020', 'Olvera Toch', 'Howard Miller 13-3/4" Diameter Brushed Chrome Round Wall Clock', 10),
 ('25-06-2020', 'Dianna Vittorini', 'AmazonBasics 3-Button USB Wired Mouse', 16),
 ('07-06-2022', 'Vivek Sundaresam', 'SANFORD Liquid Accent Tank-Style Highlighters', 7),
 ('06-05-2022', '

In [22]:
result = session.query(Products.product_name).all()

nama_barang = 'xerox'
sort_desc = 'false'

produk = [i[0] for i in result]
cari_barang = []
for i in produk:
    if nama_barang.lower() in i.lower():
         cari_barang.append(i)
            
if sort_desc==None:
    pass
elif sort_desc.lower() == 'true':
    cari_barang.sort(reverse=False)
elif sort_desc.lower() == 'false':
    cari_barang.sort(reverse=True)
cari_barang

2022-12-23 05:46:12,444 INFO sqlalchemy.engine.Engine SELECT products.product_name AS products_product_name 
FROM products
2022-12-23 05:46:12,445 INFO sqlalchemy.engine.Engine [cached since 660s ago] {}


['Xerox WorkCentre 6505DN Laser Multifunction Printer',
 'Xerox Color Copier Paper, 11" x 17", Ream',
 'Xerox Blank Computer Paper',
 'Xerox 4200 Series MultiUse Premium Copy Paper (20Lb. and 84 Bright)',
 'Xerox 232',
 'Xerox 231',
 'Xerox 230',
 'Xerox 23',
 'Xerox 229',
 'Xerox 228',
 'Xerox 227',
 'Xerox 226',
 'Xerox 225',
 'Xerox 224',
 'Xerox 223',
 'Xerox 222',
 'Xerox 221',
 'Xerox 220',
 'Xerox 22',
 'Xerox 219',
 'Xerox 218',
 'Xerox 217',
 'Xerox 216',
 'Xerox 215',
 'Xerox 214',
 'Xerox 213',
 'Xerox 212',
 'Xerox 211',
 'Xerox 210',
 'Xerox 21',
 'Xerox 209',
 'Xerox 208',
 'Xerox 207',
 'Xerox 206',
 'Xerox 205',
 'Xerox 204',
 'Xerox 203',
 'Xerox 202',
 'Xerox 201',
 'Xerox 2000',
 'Xerox 200',
 'Xerox 20',
 'Xerox 2',
 'Xerox 1999',
 'Xerox 1998',
 'Xerox 1997',
 'Xerox 1996',
 'Xerox 1995',
 'Xerox 1994',
 'Xerox 1993',
 'Xerox 1992',
 'Xerox 1991',
 'Xerox 1990',
 'Xerox 199',
 'Xerox 1989',
 'Xerox 1988',
 'Xerox 1987',
 'Xerox 1986',
 'Xerox 1985',
 'Xerox 1984',


In [1]:
import pandas as pd
import random
random.seed(7)
from datetime import datetime, timedelta
import urllib.parse
from dotenv import dotenv_values

import sqlalchemy
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base

from fastapi import FastAPI, File, UploadFile
import uvicorn
import nest_asyncio
nest_asyncio.apply()

params = {'db':dotenv_values('TugasAkhir.env')}

db_user = params['db']['MYSQL_USERNAME']
db_password = urllib.parse.quote_plus(params['db']['MYSQL_PASSWORD'])
db_host = params['db']['MYSQL_HOST']
db_port = params['db']['MYSQL_PORT']
db_name = params['db']['MYSQL_DB']

# Buat koneksi ke database MySQL
SQLALCHEMY_DATABASE_URL = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(SQLALCHEMY_DATABASE_URL, echo=True, pool_pre_ping=True)

# Membuat tanggal acak
def random_tanggal():
    # Set the start and end dates
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2022, 12, 17)
    # Generate a random date between the start and end dates
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    # Convert the random date to a string in the DD--MM-YYYY format
    formatted_date = random_date.strftime("%d-%m-20%y")
    return formatted_date

# Buat base class untuk ORM
Base = declarative_base()

try:
    # Buat tabel products
    class Products(Base):
        __tablename__ = 'products'
        product_id = Column(Integer, nullable=False)
        product_name = Column(String(50), primary_key=True)
        category = Column(String(50), nullable=False)
        sub_category = Column(String(50), nullable=False)

    # Buat tabel users
    class Users(Base):
        __tablename__ = 'users'
        customer_id = Column(Integer, nullable=False)
        name = Column(String(50), primary_key=True)
        city = Column(String(50), nullable=False)
        state = Column(String(50), nullable=False)
        postal = Column(Integer, nullable=False)

    # Buat tabel purchase
    class Purchase(Base):
        __tablename__ = 'purchase'
        name = Column(String(50), nullable=False, primary_key=True)
        date = Column(DateTime, nullable=False)
        product_name = Column(String(50), nullable=False, primary_key=True)
        quantity = Column(Integer, nullable=False)

    Base.metadata.create_all(engine)
except Exception as e:
    print(e)        

app = FastAPI()

def tabel_sql_jadi(users_csv,products_csv):
    df_users = pd.read_csv(users_csv.file)
    df_products = pd.read_csv(products_csv.file, sep=';', encoding='latin-1')
    df_products.columns = ['product_id', 'product_name', 'category', 'sub_category']

    n = 1000 #banyaknya kolom tabel purchase

    df_purchase = pd.DataFrame({'date':[random_tanggal() for i in range(n)],
                         'name':[random.choices(df_users['name'])[0] for i in range(n)], 
                         'product_name':[random.choices(df_products['product_name'])[0] for i in range(n)],
                         'quantity':[random.randint(1,25) for i in range(n)]
                        })
    
    # Insert data ke tabel products
    df_products.to_sql('products', engine, if_exists='replace', index=False)

    # Insert data ke tabel users
    df_users.to_sql('users', engine, if_exists='replace', index=False)

    # Insert data ke tabel purchase
    df_purchase.to_sql('purchase', engine, if_exists='replace', index=False)
    
# Buat Session
Session = sessionmaker(bind=engine)
session = Session()
conn = engine.connect()

# Menampilkan isi dari tabel (users, products, purchase)
@app.post("/tabel/{nama_tabel}")
async def tampilkan_isi_tabel(users_csvfile: UploadFile = File(...), 
                          products_csvfile: UploadFile = File(...),
                          nama_tabel = str
                         ):
    tabel_sql_jadi(users_csvfile,products_csvfile)
    
    return conn.execute(f'select * from {nama_tabel.lower()}').all()

# Menampilkan semua barang dengan urutan ascending (None, True, False)
@app.post("/barang")
async def tampilkan_barang(users_csvfile: UploadFile = File(...), 
                          products_csvfile: UploadFile = File(...),
                          sort_desc = None
                         ):
    tabel_sql_jadi(users_csvfile,products_csvfile)
    
    result = session.query(Products.product_name).all()
    session.commit()
    
    produk = [i[0] for i in result]
    if sort_desc==None:
        pass
    elif sort_desc.lower() == 'true':
        produk.sort(reverse=False)
    elif sort_desc.lower() == 'false':
        produk.sort(reverse=True)
    return produk

# Menampilkan semua barang yang bernama <nama_barang> dengan urutan ascending (None, True, False)
@app.post("/search_barang/{nama_barang}")
async def tampilkan_barang(users_csvfile: UploadFile = File(...), 
                          products_csvfile: UploadFile = File(...),
                          nama_barang = '',
                          sort_desc = None
                         ):
    tabel_sql_jadi(users_csvfile,products_csvfile)
    
    result = session.query(Products.product_name).all()
    session.commit()

    produk = [i[0] for i in result]
    cari_barang = []
    for i in produk:
        if nama_barang.lower() in i.lower():
             cari_barang.append(i)

    if sort_desc==None:
        pass
    elif sort_desc.lower() == 'true':
        cari_barang.sort(reverse=False)
    elif sort_desc.lower() == 'false':
        cari_barang.sort(reverse=True)
    return cari_barang



if __name__ == "__main__":
    uvicorn.run(app)
    


2022-12-23 06:02:10,606 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-12-23 06:02:10,608 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 06:02:10,611 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-12-23 06:02:10,612 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 06:02:10,615 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-12-23 06:02:10,616 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 06:02:10,618 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-23 06:02:10,619 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 06:02:10,620 INFO sqlalchemy.engine.Engine [generated in 0.00114s] {'table_schema': 'penjualan_uzu', 'table_name': 'products'}
2022-12-23 06:02:10,622 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 06:02:1

INFO:     Started server process [10404]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:62706 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:62706 - "POST /ajax.php?v=1671750134525 HTTP/1.1" 404 Not Found
2022-12-23 06:02:20,541 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 06:02:20,542 INFO sqlalchemy.engine.Engine [cached since 9.924s ago] {'table_schema': 'penjualan_uzu', 'table_name': 'products'}
2022-12-23 06:02:20,546 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 06:02:20,547 INFO sqlalchemy.engine.Engine [cached since 9.929s ago] {'table_schema': 'penjualan_uzu', 'table_name': 'products'}
2022-12-23 06:02:20,552 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `penjualan_uzu`
2022-12-23 06:02:20,553 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 06:02:20,564 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `pro

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [10404]
